## PGX - Phuture GameFi Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.1e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["gaming"]

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                -0.12,
                                100000,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens]
    )
liquidity

Excluding vanar-chain, pricing data available only for 174 < 180 days
Excluding zentry, pricing data available only for 33 < 180 days
Excluding xai-blockchain, pricing data available only for 141 < 180 days
Excluding sappy-seals-pixl, pricing data available only for 150 < 180 days
Excluding gamegpt, pricing data available only for 138 < 180 days
Excluding bag, pricing data available only for 89 < 180 days
gala on ethereum returned an invalid API response
gala recorded no valid response across any platform
ronin not supported
harmony-shard-0 not supported
energi not supported
harmony-shard-0 not supported
energi not supported
energi not supported
harmony-shard-0 not supported
energi not supported
solana not supported
Manually calculating slippage for stepn on ethereum
ronin not supported
harmony-shard-0 not supported
energi not supported
klay-token not supported
aptos not supported
okex-chain not supported
Manually calculating slippage for alien-worlds on ethereum
wax not supported
ener

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain
id,,,,,,,,,,
illuvium,NaN,NaN,NaN,NaN,-0.009388,NaN,ethereum,-0.009388,-0.009388,ethereum
echelon-prime,NaN,NaN,-0.045652,NaN,-0.013027,NaN,base,-0.013027,-0.045652,ethereum
floki,NaN,NaN,NaN,-1.000000,-0.013722,NaN,ethereum,-0.013722,-0.013722,ethereum
metahero,NaN,NaN,NaN,-0.017531,NaN,NaN,binance-smart-chain,-0.017531,-0.017531,binance-smart-chain
beam-2,NaN,NaN,NaN,-0.194570,-0.018393,NaN,ethereum,-0.018393,-0.018393,ethereum
...,...,...,...,...,...,...,...,...,...,...
cocos-bcx,NaN,NaN,NaN,-0.999999,-0.999825,NaN,None,-0.999825,-0.999825,ethereum
volt-inu-2,NaN,NaN,NaN,-1.000000,-1.000000,-1.0,None,-1.000000,-1.000000,binance-smart-chain
phantasma,NaN,NaN,NaN,-1.000000,NaN,NaN,None,-1.000000,-1.000000,binance-smart-chain


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
immutable-x,IMX,3426651258,0.302033,78,0xf57e7e7c23978c3caec3c3548e3d615c346e79ff,ethereum
floki,FLOKI,2983671221,0.302033,78,0xcf0c122c6b73ff809c693db761e7baebe62b6a2e,ethereum
beam-2,BEAM,1423046991,0.145836,38,0x62d0a8458ed7719fdaf978fe5929c6d342b0bfce,ethereum
the-sandbox,SAND,1016645550,0.104187,27,0xbbba073c31bf03b8acf7c28ef0738decf3695683,polygon-pos
illuvium,ILV,588229494,0.060283,16,0x767fe9edc9e0df98e07454847909b5e959d7ca0e,ethereum
magic,MAGIC,206479666,0.021160,5,0x539bde0d7dbd336b79148aa742883198bbf60342,arbitrum-one
wilder-world,WILD,188040090,0.019271,4,0x2a3bff78b79a009976eea096a51a948a3dc00e34,ethereum
bluzelle,BLZ,111919728,0.011470,2,0x5732046a883704404f284ce41ffadd5b007fd668,ethereum
mobox,MBOX,94250933,0.009659,2,0x3203c9e46ca618c8c1ce5dc67e7e9d75f5da2377,binance-smart-chain
